In [1]:
import segmentation_models as sm
import os
import sys

Using TensorFlow backend.
/home/niketrp/anaconda3/envs/pharedox/lib/python3.7/site-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [3]:
sys.path.append('../')
from pharynx_redox import (
    pharynx_io as pio,
    image_processing as ip,
    experiment,
    plots,
    profile_processing as pp,
    utils,
    data_analysis as da,
    constants
)

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, transform

In [4]:
try:
    os.mkdir('../data/segmentation_training_data/cropped_segmentation')
except FileExistsError:
    pass

In [5]:
def crop(s_img, fl_img, crop_amount=32):
    # Padding the images with 32 cells down and left
    s_img = np.pad(s_img, ((0,32),(32,0)), 'constant')
    fl_img = np.pad(fl_img, ((0,32),(32,0)), 'constant')
   
    # Getting the center of pharynx based on padded image
    l = measure.label(s_img)
    props = measure.regionprops(l)
    center = props[0].centroid
    
    # Cropping images around center
    upper_bound = int(center[0] - crop_amount)
    lower_bound = int(center[0] + crop_amount)
    left_bound = int(center[1] - crop_amount)
    right_bound = int(center[1] + crop_amount)
    s_img_cropped = s_img[upper_bound:lower_bound, left_bound:right_bound]
    f_img_cropped = fl_img[upper_bound:lower_bound, left_bound:right_bound]
    
    cropped_imgs = [s_img_cropped, f_img_cropped]
    return cropped_imgs


In [30]:
orig_segment_dir = '/home/niketrp/apfeld/pharynx_redox/data/segmentation_training_data/original_segmentation/'
cropped_segment_dir = '/home/niketrp/apfeld/pharynx_redox/data/segmentation_training_data/cropped_segmentation/'

seg_to_img = {}

for segment_directory in os.listdir(orig_segment_dir):
    for image_name in os.listdir(orig_segment_dir + segment_directory):
        if image_name.endswith('tif') and not image_name.startswith('seg'):
            if os.path.exists(orig_segment_dir + segment_directory + '/seg-' + image_name):
                img = orig_segment_dir + segment_directory + '/' + image_name
                seg = orig_segment_dir + segment_directory + '/seg-' + image_name
                seg_to_img[seg] = img
                
name_to_imgpairs = {}

for key,value in seg_to_img.items():
    segs = pio.load_tiff_from_disk(key)
    imgs = pio.load_tiff_from_disk(value)
    bound = ( len(segs) if len(segs) < len(imgs) else len(imgs) )
    all_cropped = []
    for i in range(0, bound - 1):
        seg = segs[i]
        img = imgs[i]
        cropped = crop(seg, img)
        all_cropped.append(cropped)
    name_to_imgpairs[key.rsplit('/', 1)[-1]] = all_cropped
    
# print(name_to_imgpairs)http://localhost:8888/notebooks/segmenation_models/segmentation_model.ipynb#

for key,value in name_to_imgpairs.items():
    top_dir = (cropped_segment_dir + key)[:-4]
    try: 
        os.mkdir(top_dir)
    except FileExistsError:
        pass
    
    for idx, pair in enumerate(value):
        pair_folder = top_dir + "/" + str(idx) 
        try: 
            os.mkdir(pair_folder)
        except FileExistsError:
            pass
        
        
        plt.imsave(pair_folder + "/seg.jpg", pair[0])
        plt.imsave(pair_folder + "/fl.jpg", pair[1])

       # plt.imshow(pair[0])
        #plt.imshow(pair[1])
#         plt.imshow(all_cropped[20][0])
#         plt.savefig(pait_folder + '/' + '0.png')
        #image = plt.imshow()
       # plt.imsave(pair_folder + "/" + "mask.jpg", all_cropped[20][0])
        
        

    
# plt.imshow(all_cropped[119][1])


